In [1]:
import sys

sys.path.append("/home/michael/pycharm/pycharm-2019.2.3/debug-eggs/pydevd-pycharm.egg")

import pydevd_pycharm
pydevd_pycharm.settrace('192.168.1.200', port=2222, stdoutToServer=True, stderrToServer=True)




sys.path.append('/home/michael/jupyter/local-packages')
from localUtilities.webScrape import getBarChartData as companyInfo
from localUtilities.webScrape import getBarChartOptionsSelenium as companyOptions
import pandas as pd

#   not 100% clear which variables this supresses
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

aStock = 'BAC'
#
stockInfo = companyInfo.getCompanyStockInfo(aStock)
stockOverview = companyInfo.getCompanyOverview(aStock)
stockFundamentals = companyInfo.getCompanyFundamentals(aStock)
aText, stockRatings = companyInfo.getCompanyRatings(aStock)
theExpiryDateText = '2020-05-29-w'
callOptions, putOptions, expiryText = companyOptions.scrapeCompanyOptionData(aStock, theExpiryDateText)

In [ ]:
putOptions


In [ ]:

callOptions.rename(column={'Strike', 'Last', '%FromLast', 'Bid',
                           'Midpoint', 'Ask', 'Change', '%Chg', 'IV', 'Volume', 'OpenInt', 'Time'}, inplace=True)


In [ ]:
startday = '2020-05-29'

theBaseCompaniesDirectory = '/home/michael/jupyter/earningDateData/Companies/'
csvSuffix = '.csv'
excelSuffix = '.xlsx'

# Setup Excel output file
companyEarningsWeek = theBaseCompaniesDirectory + startday + '/rawCSV/'
outExcelFile = theBaseCompaniesDirectory + startday + '/' + aStock + '_SummaryWeekOf-' + startday + excelSuffix


sheetIsFundamentals = 'Fundamentals'
sheetRowStart = 7
sheetColStart = 3

writer = pd.ExcelWriter(outExcelFile, engine='xlsxwriter')
# stockInfo.to_excel(writer, sheet_name='Fundamentals')

fundamentalsWorkbook = writer.book
fmt = fundamentalsWorkbook.add_format()

stockInfoTransposed = stockInfo.T
stockFundamentalsTranposed = stockFundamentals.T

# Add the Ratings next
stockRatings.to_excel(writer, sheet_name= sheetIsFundamentals,
                                    startrow=3, startcol= sheetColStart)
# Add the fundamentals
stockInfo.to_excel(writer, sheet_name= sheetIsFundamentals,  startrow=sheetRowStart, startcol=sheetColStart)
stockFundamentalsTranposed.to_excel(writer, sheet_name= sheetIsFundamentals, startrow=sheetRowStart,
                                    startcol= sheetColStart + 3)
stockOverview.to_excel(writer, sheet_name= sheetIsFundamentals,
                                    startrow=sheetRowStart, startcol= sheetColStart + 6)

# Add the text info on Expiry and Recommendations
fundamentalsWorkbookSheet = writer.sheets['Fundamentals']
fundamentalsWorkbookSheet.write(2,1, aText)
fundamentalsWorkbookSheet.write(1,1, expiryText)

callOptions.to_excel(writer, sheet_name= 'Call Options',
                                    startrow=2, startcol= sheetColStart)
putOptions.to_excel(writer, sheet_name= 'Put Options',
                                    startrow=2, startcol= sheetColStart)


# Save excel
writer.save()

In [ ]:
callOptions